<a href="https://colab.research.google.com/github/matthewchung74/inference_nbs/blob/main/yolov5/pretrained_cv2_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

based on https://github.com/ultralytics/yolov5/issues/36

In [ ]:
pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [ ]:
import cv2
import torch
from PIL import Image

model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

In [ ]:
!pip install -q git+https://github.com/matthewchung74/inference_params.git

In [ ]:
import torch.nn.functional as F
import cv2
import torch
import io
import base64
import json

from inference_params.inference_params import inference_test, FieldType, inference_predict

# image input will be the label for the app input
input = {"image input": FieldType.OpenCV}
# result will be the rendered image
output = {"result image": FieldType.OpenCV, "num boxes": FieldType.Text}

@inference_predict(input=input, output=output)
def predict(params):
    img = params['image input']
    results = model(img)  # inference
    results.imgs # array of original images (as np array) passed to model for inference
    results.render()  # updates results.imgs with boxes and labels
    cv2_image = cv2.cvtColor(results.imgs[0], cv2.COLOR_BGR2RGB)
    num_boxes = f"number of boxes {len(results.pandas().xyxy[0])}"
    return {"result image": cv2_image, "num boxes":num_boxes}

In [ ]:
from inference_params.inference_params import in_colab

if in_colab():
    from google.colab.patches import cv2_imshow

    torch.hub.download_url_to_file('https://github.com/ultralytics/yolov5/releases/download/v1.0/bus.jpg', 'bus.jpg')
    img = cv2.imread('bus.jpg')[:, :, ::-1] 

    from inference_params.inference_params import in_colab
    from matplotlib.pyplot import imshow
    import numpy as np

    from google.colab.patches import cv2_imshow
    result, duration = predict({'image input': img})
    result_image = result['result image']
    num_boxes = result['num boxes']
    print(num_boxes)
    cv2_imshow(result_image)

In [ ]:
from inference_params.inference_params import in_colab

if in_colab():
    inference_test(predict_func=predict, params={'image input': img})